In [1]:
import pandas as pd
import numpy as np
import re

# Load Data

In [26]:
df = pd.read_csv("clean_data2.csv",delimiter=",",low_memory=False)
print df.shape
df.head(10)

(11326, 1450)


CMI:75 F3a) Epilepsy: Ever suffered  \
0                                  3.0   
1                                  3.0   
2                                  3.0   
3                                  3.0   
4                                  3.0   
5                                  3.0   
6                                  3.0   
7                                  3.0   
8                                  3.0   
9                                  3.0   

   1P Child 8 years-old at NCDS1 Parental  \
0                                     NaN   
1                                     NaN   
2                                     NaN   
3                                     NaN   
4                                     NaN   
5                                     1.0   
6                                     NaN   
7                                     NaN   
8                                     NaN   
9                                     NaN   

   1P Family moves since child's birth  1P Moves out of local area  \
0                                  1.0                         3.0   
1                                  3.0                         2.0   
2                                  3.0                         3.0   
3                                  NaN                         NaN   
4                                 12.0                         2.0   
5                                  0.0                         1.0   
6                                  0.0                         1.0   
7                                  0.0                         1.0   
8                                  2.0                         2.0   
9                                  1.0                         3.0   

   1P Informant's relationship to child  \
0                                   1.0   
1                                   1.0   
2                                   1.0   
3                                   NaN   
4                                   2.0   
5                                   1.0   
6                                   1.0   
7                                   1.0   
8                                   1.0   
9                                   1.0   

   1P No. kids undr 21 hhld,inc liv away  \
0                                    3.0   
1                                    2.0   
2                                    2.0   
3                                    NaN   
4                                    2.0   
5                                    8.0   
6                                    4.0   
7                                    6.0   
8                                    2.0   
9                                    4.0   

   1P Number of people in the household  1P Child's posn hhld ,inc those away  \
0                                   5.0                                   1.0   
1                                   4.0                                   2.0   
2                                   4.0                                   2.0   
3                                   NaN                                   NaN   
4                                   4.0                                   1.0   
5                                   NaN                                   6.0   
6                                   6.0                                   3.0   
7                                   8.0                                   4.0   
8                                   4.0                                   1.0   
9                                   6.0                                   1.0   

   1P Relationship person acting as mother  \
0                                      2.0   
1                                      2.0   
2                                      2.0   
3                                      NaN   
4                                      2.0   
5                                      2.0   
6                                      2.0   
7                                      2.0   
8       

In [28]:
#I have already deleted these relevant predictors
drop = ["1M Epileptic condition","1M Reason for Special Education MC3:3","1M Reason for Special Education MC2:3","1M Reason for Special Education MC1:3","2P Has child had epilepsy attacks-MC 3:3","2P Has child had epilepsy attacks-MC 2:3",
        "2P Has child had epilepsy attacks-MC 1:3","2M Reason for special education - MC1:3","2M Reason for special education - MC2:3",
        "2M Reason for special education - MC3:3","2P Age at most recent epilepsy attack","2P Age at 1st epilepsy attack"
       ,"3P Nature of child-s disability-MC 1:5","3P Nature of child-s disability-MC 2:5","3P Nature of child-s disability-MC 4:5",
        "3P Nature of child-s disability-MC 5:5","3M Category of child's handicap MC1:3","3M Category of child's handicap MC2:3",
       "3M Category of child's handicap MC3:3","3M Reason for hosp admiss last 12 mnths","3M Reason hosp outpatient last yr"
       ,"3M Epilepsy","3M CNS condition,other than epilepsy","CMI:75 F3b) Epilepsy: Suffered in last12 months","CMI:75 F3c) Epilepsy: Seen doctor in last 12 months" ]

In [32]:
df["3P Reason for absence from school-MC 1:6"].unique()

array([ nan,   1.,   2.,   4.,   6.,   7.,   9.,   5.,  12.,  11.,   8.,
        10.,   3.])

# **Instructions**:
The dataset is a combination (merge by ID number) of sweep 0-3 and sweep 5. There are in total 11326 samples with 1474 predictors and one response at the **first** row. I manually filtered good variables related to predicting epilepsy:

### sweep0-3:
- 1P NCDS1 (1965) Parental Questionnaire 
- 1PD NCDS1 (1965) Parental Questionnaire (Derived Variable) 
- 1M NCDS1 (1965) Medical Questionnaire 
- ~~1S NCDS1 (1965) Educational Assessment~~ 
- 1D NCDS1 (1965) Derived Variable 
- 2P NCDS2 (1969) Parental Questionnaire 
- 2M NCDS2 (1969) Medical Questionnaire 
- 3P NCDS3 (1974) Parental Interview 
- ~~3S NCDS3 (1974) Educational Questionnaire~~ 
- 3M NCDS3 (1974) Medical Examination 

### sweep6:
Health variables from NCDS official website.  
http://www.cls.ioe.ac.uk/datadictionary/page.asp?section=000100010001001000010007&sectionTitle=G+%2D+Health

**Response variable: N503930	CMI:75 F3a) Epilepsy: Ever suffered**:

    Pos. = 1140	Variable = n503930	Variable label = CMI:75 F3a) Epilepsy: Ever suffered  
    This variable is  numeric, the SPSS measurement level is ORDINAL 

    Value label information for n503930  
    Value = 1.0	Label = Yes  
    Value = 2.0	Label = Only when pregnant  
    Value = 3.0	Label = No  

#  Next Step: Transform the data and do classification

In [29]:
target = "CMI:75 F3a) Epilepsy: Ever suffered"
print "with epilepsy: ", len(df[target][df[target] == 1])
print "with epilepsy only when pregnant: ",len(df[target][df[target] == 2])
print "without epilepsy: ", len(df[target][df[target] == 3])

with epilepsy:  141
with epilepsy only when pregnant:  22
without epilepsy:  11163


**Ideas**: For the whole population part, maybe delete samples **with epilepsy only when pregnant** and do a binary classification. We can further target at women to do three class classification. :)